In [1]:
using OrdinaryDiffEq, ForwardDiff, Distributions, RandomNumbers, NPZ

In [2]:
function initial_sampling(Eb,w1,mb)
    nums=rand(1)*2*pi
    x1=cos(nums[1])*sqrt(Eb/(0.5*mb*w1*w1))
    y1=sin(nums[1])*sqrt(2*mb*Eb)
    return x1,y1
end

function bolz_energy(a)
    return rand(Truncated(Exponential(1), 0., 10))
end
function bolz_freq(a)
    return rand()*0.4324555320336759+0.63
end
N=0

initial_position = [0.1, 0.,0.,0.,0.,0.,0.,0.,0.]
initial_momentum=[0. ,0.1,0.,0.,0.,0.,0.,0.,0.]
numeros=[2,4,6,8,10,16,20,26,30,36,40,50,60,80,100]
#numeros=[4,6]
promedios=100
solu=0
w=0
Es=0
count=zeros(size(numeros)[1])
for ii in 1:size(numeros)[1]
    println(ii)
    ii=Int(ii)
        
for jj in 1:promedios
    jj=Int(jj)
N=numeros[ii]
initial_position=ones(N+1)
initial_momentum=ones(N+1)

a=0.2
b=0.01

m=[1.,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1]
w=[0.,0.8,0.8,0.8,0.8,0.8,0.8,0.8,0.8]
g=[0.,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1]

X0=17.    
P0=1.

golden=(1+sqrt(big(5)))/2
Es=zeros(N+1)

time=10000.
#time=1.
tspan=(0., time)
#fig=figure()
w=zeros(N+1)
g=zeros(N+1)
E=zeros(N+1)
m=ones(N+1) .*0.1
m[1]=1.
w=w .* 0. .+ 1.
w=(bolz_freq.(w))
Es=(bolz_energy.(Es))



#println(w)
W=maximum(w)
g=g .* 0. .+ 0.3/N

dts=0.01*2. *pi/W
#Hsys(q,p) = p[1]^2. /(2. *m[1]) +m[1]*w[1]^2. *q[1]^2. /2. +0.5* -q[1]*g[2]*q[2]+p[2]^2. /(2. *m[2])+m[2]*w[2]^2. *q[2]^2. /2. + q[1]^2. *g[2]^2. /(2. *m[2]*w[2]^2.) -q[1]*g[3]*q[3]+p[3]^2. /(2. *m[3])+m[3]*w[3]^2. *q[3]^2. /2. + q[1]^2. *g[3]^2. /(2. *m[3]*w[3]^2.) -q[1]*g[4]*q[4]+p[4]^2. /(2. *m[4])+m[4]*w[4]^2. *q[4]^2. /2. + q[1]^2. *g[4]^2. /(2. *m[4]*w[4]^2.) -q[1]*g[5]*q[5]+p[5]^2. /(2. *m[5])+m[5]*w[5]^2. *q[5]^2. /2. + q[1]^2. *g[5]^2. /(2. *m[5]*w[5]^2.) -q[1]*g[6]*q[6]+p[6]^2. /(2. *m[6])+m[6]*w[6]^2. *q[6]^2. /2. + q[1]^2. *g[6]^2. /(2. *m[6]*w[6]^2.) -q[1]*g[7]*q[7]+p[7]^2. /(2. *m[7])+m[7]*w[7]^2. *q[7]^2. /2. + q[1]^2. *g[7]^2. /(2. *m[7]*w[7]^2.) -q[1]*g[8]*q[8]+p[8]^2. /(2. *m[8])+m[8]*w[8]^2. *q[8]^2. /2. + q[1]^2. *g[8]^2. /(2. *m[8]*w[8]^2.)-q[1]*g[9]*q[9]+p[9]^2. /(2. *m[9])+m[9]*w[9]^2. *q[9]^2. /2. + q[1]^2. *g[9]^2. /(2. *m[9]*w[9]^2.) 
function Hsys(q,p,N0)
    result=p[1]^2. /(2. *m[1]) - a*q[1]^2. /2. +b*q[1]^4. /4.
    for i in 1:N0
        result +=  q[1]*g[i+1]*q[i+1]+p[i+1]^2. /(2. *m[2])+m[i+1]*w[i+1]^2. *q[i+1]^2. /2.
    end
    return result
end
pdot(dp,p,q,params,t) = ForwardDiff.gradient!(dp, q->-Hsys(q, p,N), q)
qdot(dq,p,q,params,t) = ForwardDiff.gradient!(dq, p-> Hsys(q, p,N), p)



initial_position=initial_position .* 0.
initial_momentum=initial_momentum .* 0.
initial_position[1]=0.0 #xz0[jj]
initial_momentum[1]=0.0 #pz1[jj]

for i in 1:N
    i=Int(i)
    initial_position[i], initial_momentum[i]=initial_sampling(Es[i],w[i],m[i])
end

 
prob = DynamicalODEProblem(pdot, qdot, initial_momentum, initial_position, tspan)
@time sol = solve(prob, CalvoSanz4(), dt=dts)
    solu=sol

for j in 1:size(sol[N+2,:])[1]-1
    if sol[N+2,j]<0 && sol[N+2,j+1]>0
        count[ii]=count[ii]+1
    elseif sol[N+2,j]>0 && sol[N+2,j+1]<0
        count[ii]=count[ii]+1
    end
end
end

npzwrite("numeros_sin_extra.npy", numeros)
npzwrite("counts_sin_extra.npy",count)
end
println(count)


1


┌ Warning: `Truncated(d::UnivariateDistribution, l::Real, u::Real)` is deprecated, use `truncated(d, l, u)` instead.
│   caller = ip:0x0
└ @ Core :-1


 17.524582 seconds (94.41 M allocations: 3.405 GiB, 11.40% gc time)
  6.544172 seconds (73.80 M allocations: 2.379 GiB, 23.68% gc time)
  6.782833 seconds (74.01 M allocations: 2.386 GiB, 25.42% gc time)
  5.339094 seconds (68.99 M allocations: 2.224 GiB, 21.06% gc time)
  6.122871 seconds (61.09 M allocations: 1.970 GiB, 20.51% gc time)
  6.209979 seconds (66.83 M allocations: 2.155 GiB, 28.65% gc time)
  5.005719 seconds (60.73 M allocations: 1.958 GiB, 20.08% gc time)
  5.193064 seconds (58.26 M allocations: 1.878 GiB, 28.67% gc time)
  6.534858 seconds (70.27 M allocations: 2.266 GiB, 26.26% gc time)
  3.887272 seconds (50.97 M allocations: 1.643 GiB, 22.96% gc time)
  5.181924 seconds (68.93 M allocations: 2.222 GiB, 22.69% gc time)
  5.731027 seconds (71.79 M allocations: 2.315 GiB, 27.03% gc time)
  5.750382 seconds (74.49 M allocations: 2.402 GiB, 27.13% gc time)
  5.361319 seconds (72.26 M allocations: 2.330 GiB, 21.52% gc time)
  4.846389 seconds (60.37 M allocations: 1.946 G

  9.385529 seconds (115.46 M allocations: 4.440 GiB, 20.34% gc time)
 10.634729 seconds (125.42 M allocations: 4.823 GiB, 20.24% gc time)
 11.118390 seconds (125.06 M allocations: 4.810 GiB, 26.58% gc time)
  9.752282 seconds (118.18 M allocations: 4.545 GiB, 21.95% gc time)
 10.667993 seconds (119.49 M allocations: 4.595 GiB, 27.50% gc time)
  8.631903 seconds (96.91 M allocations: 3.727 GiB, 21.66% gc time)
 11.563276 seconds (126.47 M allocations: 4.864 GiB, 21.04% gc time)
 11.888435 seconds (128.65 M allocations: 4.948 GiB, 26.41% gc time)
  9.393020 seconds (127.99 M allocations: 4.922 GiB, 19.88% gc time)
  9.663313 seconds (127.92 M allocations: 4.919 GiB, 25.09% gc time)
  9.672471 seconds (127.31 M allocations: 4.896 GiB, 24.80% gc time)
  9.412062 seconds (126.45 M allocations: 4.863 GiB, 21.31% gc time)
  8.496900 seconds (116.15 M allocations: 4.467 GiB, 21.14% gc time)
  9.232930 seconds (122.82 M allocations: 4.723 GiB, 25.04% gc time)
  8.059284 seconds (111.77 M alloca

 11.928767 seconds (180.21 M allocations: 8.185 GiB, 23.42% gc time)
 12.339667 seconds (181.98 M allocations: 8.265 GiB, 24.23% gc time)
 11.371155 seconds (173.82 M allocations: 7.895 GiB, 22.00% gc time)
 10.732157 seconds (160.67 M allocations: 7.298 GiB, 22.31% gc time)
 12.618329 seconds (178.55 M allocations: 8.110 GiB, 24.38% gc time)
  9.777328 seconds (148.00 M allocations: 6.722 GiB, 20.70% gc time)
 12.478623 seconds (174.53 M allocations: 7.927 GiB, 23.61% gc time)
 14.894831 seconds (171.81 M allocations: 7.804 GiB, 22.49% gc time)
 13.290152 seconds (171.19 M allocations: 7.776 GiB, 20.86% gc time)
 14.072700 seconds (184.30 M allocations: 8.371 GiB, 23.90% gc time)
 11.554226 seconds (162.63 M allocations: 7.387 GiB, 21.16% gc time)
 13.478969 seconds (177.28 M allocations: 8.052 GiB, 25.66% gc time)
 13.190601 seconds (180.88 M allocations: 8.216 GiB, 20.78% gc time)
 14.543509 seconds (183.15 M allocations: 8.319 GiB, 27.72% gc time)
 11.935998 seconds (169.65 M alloc

 15.635442 seconds (218.92 M allocations: 11.415 GiB, 18.72% gc time)
 17.411717 seconds (238.15 M allocations: 12.418 GiB, 22.69% gc time)
 16.423993 seconds (238.89 M allocations: 12.457 GiB, 19.57% gc time)
 16.832128 seconds (234.46 M allocations: 12.226 GiB, 20.39% gc time)
 17.946987 seconds (234.73 M allocations: 12.240 GiB, 22.13% gc time)
 18.466791 seconds (222.99 M allocations: 11.628 GiB, 22.11% gc time)
 17.596843 seconds (225.67 M allocations: 11.767 GiB, 19.10% gc time)
 16.120097 seconds (202.79 M allocations: 10.574 GiB, 22.55% gc time)
 16.956399 seconds (237.31 M allocations: 12.374 GiB, 19.93% gc time)
 17.814382 seconds (235.42 M allocations: 12.275 GiB, 22.14% gc time)
 17.493666 seconds (234.69 M allocations: 12.238 GiB, 19.26% gc time)
 15.563001 seconds (207.08 M allocations: 10.798 GiB, 22.52% gc time)
 17.815963 seconds (227.60 M allocations: 11.868 GiB, 22.35% gc time)
 16.198165 seconds (225.93 M allocations: 11.781 GiB, 19.94% gc time)
 16.998557 seconds (

 29.129814 seconds (291.15 M allocations: 17.288 GiB, 22.65% gc time)
 26.254801 seconds (259.24 M allocations: 15.393 GiB, 28.10% gc time)
 25.097683 seconds (288.66 M allocations: 17.140 GiB, 22.36% gc time)
 23.094888 seconds (284.68 M allocations: 16.904 GiB, 23.02% gc time)
 24.078024 seconds (289.15 M allocations: 17.169 GiB, 23.62% gc time)
 25.193309 seconds (279.71 M allocations: 16.609 GiB, 23.58% gc time)
 22.951061 seconds (288.64 M allocations: 17.139 GiB, 23.10% gc time)
 24.224333 seconds (271.33 M allocations: 16.111 GiB, 23.02% gc time)
 25.286490 seconds (293.69 M allocations: 17.439 GiB, 25.65% gc time)
 25.255338 seconds (292.55 M allocations: 17.371 GiB, 23.02% gc time)
 26.087113 seconds (288.52 M allocations: 17.132 GiB, 21.84% gc time)
 23.777131 seconds (257.23 M allocations: 15.273 GiB, 25.96% gc time)
 24.763244 seconds (279.10 M allocations: 16.572 GiB, 24.92% gc time)
 24.533462 seconds (272.00 M allocations: 16.151 GiB, 22.46% gc time)
 26.706494 seconds (

InterruptException: InterruptException: